# YUFE University Collaboration & Publication Analysis

This project analyzes interdisciplinary collaborations and top subfields among universities using OpenAlex data.


## Setup & Imports

In [ ]:
import sys
# Logging, paths, and constants
import pandas as pd
import gc
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations
from pathlib import Path
import logging



# Configure logging
LOGS_DIR = Path("logs")
LOGS_DIR.mkdir(exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler(LOGS_DIR / "analysis.log", mode='w')
    ]
)
logger = logging.getLogger(__name__)


## Constants & Paths

In [ ]:
# Paths and filtering settings
DATA_PATH = Path("data/merged_openalex_data.csv")  # Adjust this path to your local CSV

# --- Note for Reproducibility ---
# The input CSV must contain the following columns:
# - OPENALEX_ID
# - Combined_University_Name
# - DOMAIN
# - FIELD
# - SUBFIELD
# - OPAL_YEAR
# These are necessary for subsequent filtering and analysis steps.
# The OPAL_YEAR column should contain publication years as integers.

CLEANING_DIR = Path("data/subfield_cleaning")

# Required columns for filtering
REQUIRED_COLUMNS = ['DOMAIN', 'FIELD', 'SUBFIELD', 'Combined_University_Name']

# Year range
START_YEAR = 2018
END_YEAR = 2023


## Load CSV Data and Filter by Year

In [ ]:
# Load main dataset and filter by year
try:
    df_filtered_by_year = pd.read_csv(DATA_PATH)
    df_filtered_by_year = df_filtered_by_year.query(f"{START_YEAR} <= OPAL_YEAR <= {END_YEAR}")
    logger.info(f"Loaded data with shape {df_filtered_by_year.shape} from {DATA_PATH}")
except FileNotFoundError:
    logger.error(f"File not found: {DATA_PATH}")
    df_filtered_by_year = pd.DataFrame()  # Empty fallback
except Exception as e:
    logger.exception(f"Unexpected error: {e}")
    df_filtered_by_year = pd.DataFrame()


# Load Subfield Cleaning Rules

In [ ]:
# Load subfield cleaning Excel files
excel_files = list(CLEANING_DIR.glob("*.xlsx"))
removal_list = []

for file in excel_files:
    try:
        temp_df = pd.read_excel(file)
        if set(REQUIRED_COLUMNS).issubset(temp_df.columns):
            removal_list.append(temp_df[REQUIRED_COLUMNS])
            logger.info(f"Valid cleaning file loaded: {file.name}")
        else:
            logger.warning(f"Skipping {file.name}: missing required columns.")
    except Exception as e:
        logger.error(f"Error reading {file.name}: {e}")

# Combine into one DataFrame
removal_df = pd.concat(removal_list, ignore_index=True) if removal_list else pd.DataFrame(columns=REQUIRED_COLUMNS)

## Apply Cleaning Filter of Irrelevant Subfields (specified by some universities)

In [ ]:
# Remove matching subfields
if not df_filtered_by_year.empty:
    df_filtered = (
        df_filtered_by_year
        .merge(removal_df, on=REQUIRED_COLUMNS, how='left', indicator=True)
        .query('_merge == "left_only"')
        .drop(columns=['_merge'])
    )
    logger.info(f"After subfield filtering: {df_filtered.shape[0]} rows remaining.")
else:
    df_filtered = pd.DataFrame()
    logger.warning("No data to filter due to previous errors.")

# Free memory
del df_filtered_by_year
del removal_df
gc.collect()
logger.info("Memory cleaned.")

# Data Visualisation

## Visualize Total Unique Publications per University

In [ ]:
## Visualize Total Unique Publications per University

import matplotlib.pyplot as plt

# --- Deduplicate publication-university pairs across fields and subfields---
try:
    unique_pub_uni_df = df_filtered.drop_duplicates(subset=["OPENALEX_ID", "Combined_University_Name"])
    logger.info(f"Unique publication-university pairs: {len(unique_pub_uni_df)}")
except Exception as e:
    logger.exception("Error during deduplication.")
    unique_pub_uni_df = pd.DataFrame()

# --- Aggregate: Count unique publications per university ---
try:
    uni_publication_counts_df = (
        unique_pub_uni_df.groupby("Combined_University_Name", as_index=False)
        .size()
        .rename(columns={"size": "Total_Unique_Publications"})
        .sort_values("Total_Unique_Publications", ascending=False)
    )
    logger.info("Aggregated publication counts per university.")
except Exception as e:
    logger.exception("Error during aggregation.")
    uni_publication_counts_df = pd.DataFrame()

# --- Plot ---
try:
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(
        uni_publication_counts_df["Combined_University_Name"],
        uni_publication_counts_df["Total_Unique_Publications"],
        color="royalblue"
    )

    # Formatting
    ax.set_title("Total Unique Publications per University")
    ax.set_xlabel("University")
    ax.set_ylabel("Total Publications")
    ax.set_xticks(range(len(uni_publication_counts_df)))
    ax.set_xticklabels(uni_publication_counts_df["Combined_University_Name"], rotation=90)
    ax.grid(axis="y", linestyle="--", alpha=0.7)

    plt.tight_layout()
    plt.show()
    logger.info("Bar plot generated successfully.")
except Exception as e:
    logger.exception("Error while plotting publication counts.")

# --- Cleanup memory ---
del unique_pub_uni_df
del uni_publication_counts_df
gc.collect()
logger.info("Memory cleaned after visualization.")


## Create Aggregated DataFrame for Top Subfields per University

In [ ]:
# Group and rank subfields per university/domain
try:
    # --- Group and rank subfields ---
    df_top_subfields_base  = (
        df_filtered
        .groupby(['Combined_University_Name', 'DOMAIN', 'FIELD', 'SUBFIELD'], as_index=False)
        .size()
        .rename(columns={'size': 'publication_count'})
    )

    df_top_subfields_base ['rank'] = (
        df_top_subfields_base
        .groupby(['Combined_University_Name', 'DOMAIN'])['publication_count']
        .rank(method='dense', ascending=False)
    )

    logger.info(f"Ranked subfields for {df_top_subfields_base ['Combined_University_Name'].nunique()} universities.")
except Exception as e:
    logger.exception("Error during subfield ranking.")
    df_top_subfields_base  = pd.DataFrame()



In [ ]:
# For Sunburst (Top 3)
df_top3_subfields = df_top_subfields_base [df_top_subfields_base ['rank'] <= 3].copy()

# For Horizontal Bar Chart (Top 15)
df_top15_subfields = df_top_subfields_base [df_top_subfields_base ['rank'] <= 15].copy()

# For later use (Top 5)
df_top5_subfields = df_top_subfields_base [df_top_subfields_base ['rank'] <= 5].copy()

gc.collect()


## Extra: Collaboration analysis tables, Median subfields

In [ ]:
import pandas as pd
from pathlib import Path

# ==============================
# SETTINGS
# ==============================
OUTPUT_DIR = Path("collaboration_tables")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ==============================
# STEP 1: Compute median publications per subfield across all 10 YUFE universities
# ==============================
df_median = (
    df_top_subfields_base
    .groupby(['DOMAIN', 'FIELD', 'SUBFIELD'], as_index=False)['publication_count']
    .median()
    .rename(columns={'publication_count': 'median_publications'})
)

# ==============================
# STEP 2: Filter subfields in the top 25% (high-value subfields)
# ==============================
top_25_cutoff = df_median['median_publications'].quantile(0.75)
df_top_subfields = df_median[df_median['median_publications'] >= top_25_cutoff]

df_with_medians = df_top_subfields_base.merge(
    df_top_subfields,
    on=['DOMAIN', 'FIELD', 'SUBFIELD'],
    how='inner'
)

# Identify all universities
all_unis = df_with_medians['Combined_University_Name'].unique()

# ==============================
# STEP 3: Create collaboration tables for each university
# ==============================
uni_tables = {}

for uni in all_unis:
    df_uni = df_with_medians[df_with_medians['Combined_University_Name'] == uni].copy()
    df_uni = df_uni[df_uni['publication_count'] >= df_uni['median_publications']]

    uni_rows = []

    for _, row in df_uni.iterrows():
        subfield = row['SUBFIELD']
        field = row['FIELD']
        domain = row['DOMAIN']
        uni_count = row['publication_count']
        median_val = row['median_publications']

        partners_df = df_with_medians[
            (df_with_medians['DOMAIN'] == domain) &
            (df_with_medians['FIELD'] == field) &
            (df_with_medians['SUBFIELD'] == subfield) &
            (df_with_medians['Combined_University_Name'] != uni) &
            (df_with_medians['publication_count'] >= median_val)
            ]

        partners_list = [
            f"{partner} ({count})"
            for partner, count in zip(partners_df['Combined_University_Name'], partners_df['publication_count'])
        ]

        uni_rows.append({
            'DOMAIN': domain,
            'FIELD': field,
            'SUBFIELD': subfield,
            f"{uni} Count": uni_count,
            'Median Publications': median_val,
            'Partner Universities': ", ".join(partners_list)
        })

    if uni_rows:
        uni_tables[uni] = pd.DataFrame(uni_rows)

# ==============================
# STEP 5: Export collaboration tables to Excel with conditional formatting
# ==============================
def safe_excel_sheet_name(name: str) -> str:
    """Create Excel-safe sheet names (<= 31 characters, no slashes)."""
    return name.replace(" ", "_").replace("/", "-")[:31]

excel_path = OUTPUT_DIR / "potential_collaborations_top25_formatted.xlsx"

with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    for uni, df_uni_table in uni_tables.items():
        df_uni_table.to_excel(writer, sheet_name=safe_excel_sheet_name(uni), index=False)

        # Apply conditional formatting
        workbook  = writer.book
        worksheet = writer.sheets[safe_excel_sheet_name(uni)]

        # Identify the column of this university's counts
        col_idx = df_uni_table.columns.get_loc(f"{uni} Count")
        col_letter = chr(ord('A') + col_idx)

        # Apply a 2-color scale
        worksheet.conditional_format(
            f"{col_letter}2:{col_letter}{len(df_uni_table)+1}",
            {'type': '2_color_scale',
             'min_color': "#FFFFFF",
             'max_color': "#63BE7B"}
        )


## Extra: Visualise the collaboration tables

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# ==============================
# Prepare data for heatmaps per domain
# ==============================
domains = df_with_medians['DOMAIN'].unique()

for domain in domains:
    # Filter data for the current domain
    df_domain = df_with_medians[df_with_medians['DOMAIN'] == domain].copy()

    # Create a "Field: Subfield" label for y-axis
    df_domain['Field_Subfield'] = df_domain['FIELD'] + ": " + df_domain['SUBFIELD']

    # Pivot table: subfields x universities
    df_pivot = df_domain.pivot_table(
        index='Field_Subfield',
        columns='Combined_University_Name',
        values='publication_count',
        fill_value=0
    )

    # Sort subfields by median across universities
    df_pivot['Median'] = df_pivot.median(axis=1)
    df_pivot = df_pivot.sort_values(by='Median', ascending=False)
    df_pivot = df_pivot.drop(columns='Median')

    # Plot heatmap
    plt.figure(figsize=(14, max(8, len(df_pivot) * 0.3)))
    sns.set(style="whitegrid")
    ax = sns.heatmap(
        df_pivot,
        annot=True,
        fmt="g",
        cmap="YlGnBu",
        linewidths=0.5,
        linecolor='gray',
        cbar_kws={'label': 'Publication Count'}
    )

    plt.title(f"Publication Counts by Subfield and University\nDomain: {domain}", fontsize=16, pad=20)
    plt.xlabel("University", fontsize=12)
    plt.ylabel("Field: Subfield", fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()



# Extra: collaboration table: scatter plot

## Sunburst Chart: Top 3 Subfields per Domain (Per University)

In [ ]:
import plotly.express as px


# --- Generate Sunburst charts ---
try:
    for university, df_uni in df_top3_subfields.groupby('Combined_University_Name'):
        fig = px.sunburst(
            df_uni,
            path=['DOMAIN', 'SUBFIELD'],
            values='publication_count',
            color='DOMAIN',
            title=f"Top 3 Subfields per Domain for {university}"
        )
        fig.update_layout(margin=dict(t=50, l=0, r=0, b=0))
        fig.show()
        logger.info(f"Sunburst chart generated for {university}.")
except Exception as e:
    logger.exception("Error generating sunburst charts.")


## Bar Chart: Top 15 Subfields per Domain (per University)

In [ ]:
import pandas as pd
import plotly.express as px
import os
import gc
from pathlib import Path
import plotly.colors as pc
import itertools

# --- Settings ---
BAR_CHART_DIR = Path("subfields_top15_barcharts")
BAR_CHART_DIR.mkdir(parents=True, exist_ok=True)

# Create a color sequence that can handle up to 26 unique fields
base_palettes = list(itertools.chain(
    pc.qualitative.Plotly,
    pc.qualitative.D3,
    pc.qualitative.Set1,
    pc.qualitative.Set2,
    pc.qualitative.Set3,
    pc.qualitative.Bold
))
if len(base_palettes) < 26:
    base_palettes = base_palettes * ((26 // len(base_palettes)) + 1)
base_palettes = base_palettes[:26]

# Get unique fields globally for consistent FIELD color mapping
unique_fields = df_top15_subfields['FIELD'].unique()
field_color_map = {field: base_palettes[i % len(base_palettes)] for i, field in enumerate(unique_fields)}

# Create a unique label for plotting: "Subfield (Field)"
df_top15_subfields['SUBFIELD_UNIQUE'] = df_top15_subfields['SUBFIELD'] + " (" + df_top15_subfields['FIELD'] + ")"

# --- Generate and save bar charts ---
for university, df_uni in df_top15_subfields.groupby("Combined_University_Name"):
    try:
        # Sort for meaningful y-axis order
        df_uni_sorted = df_uni.sort_values(['FIELD', 'publication_count'], ascending=[True, False])
        subfield_order = df_uni_sorted['SUBFIELD_UNIQUE'].tolist()

        # Create bar chart with FIELD-based colors
        fig = px.bar(
            df_uni_sorted,
            x='publication_count',
            y='SUBFIELD_UNIQUE',  # Use unique FIELD+SUBFIELD
            color='FIELD',
            color_discrete_map=field_color_map,
            orientation='h',
            title=f"Top 15 Subfields per Domain for {university}",
            labels={
                'publication_count': 'Publication Count',
                'SUBFIELD_UNIQUE': 'Subfield'
            }
        )

        fig.update_layout(
            margin=dict(t=50, l=300, r=50, b=150),
            height=1000,
            yaxis=dict(
                categoryorder='array',
                categoryarray=subfield_order,
                tickmode='array',
                tickvals=subfield_order,
                ticktext=subfield_order,
                tickfont=dict(size=10)
            )
        )

        # Save chart as HTML
        safe_filename = university.replace(" ", "_").replace("/", "-")
        filepath = BAR_CHART_DIR / f"{safe_filename}.html"
        fig.write_html(filepath)
        logger.info(f"Bar chart saved: {filepath}")

    except Exception as e:
        logger.error(f"Failed to generate chart for {university}: {e}")

# --- Cleanup ---
gc.collect()
logger.info("Top 15 subfield bar chart generation completed and memory cleaned.")


## Sunburst Chart: Top 15 Subfields per Domain (per University)

In [ ]:
import pandas as pd
import plotly.express as px
import os
import gc

# --- Settings ---
SUNBURST_DIR = Path("subfield_top15_sunburst")
SUNBURST_DIR.mkdir(parents=True, exist_ok=True)

# --- Create sunburst chart per university ---
for university, df_uni in df_top15_subfields.groupby("Combined_University_Name"):
    try:
        fig = px.sunburst(
            df_uni,
            path=['DOMAIN', 'FIELD', 'SUBFIELD'],
            values='publication_count',
            color='FIELD',
            title=f"Top 15 Subfields per Domain for {university}",
            labels={'publication_count': 'Publication Count'}
        )

        fig.update_layout(margin=dict(t=50, l=0, r=0, b=0))

        # Safe filename and save
        safe_filename = f"{university.replace(' ', '_').replace('/', '-')}.html"
        fig.write_html(SUNBURST_DIR / safe_filename)
        logger.info(f"Sunburst chart saved: {SUNBURST_DIR / safe_filename}")

    except Exception as e:
        logger.error(f"Failed to generate sunburst for {university}: {e}")

# --- Cleanup ---
gc.collect()
logger.info("Top 15 sunburst chart generation completed and memory cleaned.")

## Sunburst Chart: Top 5 Subfields per Domain (per University)

In [ ]:
import pandas as pd
import plotly.express as px
from pathlib import Path
import gc

# --- Settings ---
SUNBURST_DIR_TOP5 = Path("subfield_top5_sunburst")
SUNBURST_DIR_TOP5.mkdir(parents=True, exist_ok=True)

# --- Loop through universities and create sunburst ---
for university, df_uni in df_top5_subfields.groupby("Combined_University_Name"):
    try:
        fig = px.sunburst(
            df_uni,
            path=['DOMAIN', 'FIELD', 'SUBFIELD'],
            values='publication_count',
            color='FIELD',
            title=f"Top 5 Subfields per Domain for {university}",
            labels={'publication_count': 'Publication Count'}
        )

        fig.update_layout(margin=dict(t=50, l=0, r=0, b=0))

        # Safe filename and save
        safe_filename = f"{university.replace(' ', '_').replace('/', '-')}.html"
        fig.write_html(SUNBURST_DIR_TOP5 / safe_filename)
        logger.info(f"Top 5 Sunburst saved: {SUNBURST_DIR_TOP5 / safe_filename}")

    except Exception as e:
        logger.error(f"Failed to generate Top 5 sunburst for {university}: {e}")

# --- Cleanup ---
gc.collect()
logger.info("Top 5 sunburst chart generation completed and memory cleaned.")


## Global Sunburst Chart: Universities and Domains (All Combined)

In [ ]:

import pandas as pd
import plotly.express as px
from pathlib import Path
import os
import gc

# --- Settings ---
GLOBAL_SUNBURST_DIR = Path("global_sunburst_html")
GLOBAL_SUNBURST_DIR.mkdir(parents=True, exist_ok=True)

# --- Deduplicate publication-domain-university level ---
try:
    df_domain_level = df_filtered.drop_duplicates(subset=['OPENALEX_ID', 'Combined_University_Name', 'DOMAIN'])
    logger.info("Deduplicated domain-level publication data.")
except Exception as e:
    logger.exception("Error during deduplication at domain level.")
    df_domain_level = pd.DataFrame()

# --- Group and calculate counts and percentages ---
try:
    df_grouped = (
        df_domain_level
        .groupby(['DOMAIN', 'Combined_University_Name'], as_index=False)
        .size()
        .rename(columns={'size': 'publication_count'})
    )
    df_grouped['domain_total'] = df_grouped.groupby('DOMAIN')['publication_count'].transform('sum')
    df_grouped['percentage'] = (df_grouped['publication_count'] / df_grouped['domain_total']) * 100
    logger.info("Grouped data and calculated domain totals and percentages.")
except Exception as e:
    logger.exception("Error during grouping or percentage calculation.")
    df_grouped = pd.DataFrame()

# --- Color settings ---
domain_colors = {
    'HEALTH': '#1f77b4',
    'SOCIAL': '#ff7f0e',
    'PHYSICAL': '#2ca02c',
    'LIFE SCIENCES': '#d62728'
}

uni_colors = {
    'University of Antwerp': '#9400D3',
    'University of Rijeka': '#FF4500',
    'Nicolaus Copernicus University and affiliations': '#4682B4',
    'University of Bremen': '#008000',
    'University of Cyprus': '#FFD700',
    'Maastricht University and affiliations': '#006400',
    'Universidad Carlos III de Madrid': '#8B0000',
    'Sorbonne Nouvelle University and affiliations': '#4B0082'
}

color_map = {**domain_colors, **uni_colors}

# --- Generate Sunburst ---
try:
    fig = px.sunburst(
        df_grouped,
        path=['DOMAIN', 'Combined_University_Name'],
        values='publication_count',
        color='Combined_University_Name',
        color_discrete_map=color_map,
        hover_data={'percentage': ':.1f%'},
        title="Global View of Domains and Universities by Publication Count",
        labels={
            'DOMAIN': 'Domain',
            'Combined_University_Name': 'University',
            'publication_count': 'Publication Count'
        }
    )
    fig.update_traces(textinfo="label+percent entry")
    fig.update_layout(margin=dict(t=50, l=0, r=0, b=0))

    output_path = GLOBAL_SUNBURST_DIR / "global_sunburst.html"
    fig.write_html(output_path)
    logger.info(f"Global sunburst chart saved to: {output_path}")

except Exception as e:
    logger.exception("Error generating global sunburst chart.")

# --- Cleanup ---
del df_domain_level, df_grouped
gc.collect()
logger.info("Global sunburst chart completed and memory cleaned.")

## Bubble Scatter Plot: Publications per University and Field

In [ ]:
import plotly.express as px
from pathlib import Path
import gc

# --- Output directory and filename ---
FIELD_SCATTER_DIR = Path("field_scatter")
FIELD_SCATTER_DIR.mkdir(parents=True, exist_ok=True)
output_file = FIELD_SCATTER_DIR / "Field_Scatter_Plot.html"

try:
    # --- Deduplicate at university-field-publication level ---
    df_field_level = df_filtered.drop_duplicates(subset=['OPENALEX_ID', 'Combined_University_Name', 'FIELD'])
    logger.info(f"Deduplicated to {len(df_field_level)} university-field-publication records.")

    # --- Aggregate publication counts ---
    df_collab_field_uni = (
        df_field_level
        .groupby(['FIELD', 'Combined_University_Name'], as_index=False)
        .size()
        .rename(columns={'size': 'publication_count'})
    )
    logger.info(f"Aggregated publication counts per field and university, total {len(df_collab_field_uni)} rows.")

    # --- Create bubble scatter plot ---
    fig = px.scatter(
        df_collab_field_uni,
        x='Combined_University_Name',
        y='FIELD',
        size='publication_count',
        color='Combined_University_Name',
        labels={'publication_count': 'Number of Publications'},
        title='Field Scatter Plot Per Institute',
        size_max=30,
        hover_name='FIELD',
        hover_data={'publication_count': True}
    )

    # --- Update layout ---
    fig.update_layout(
        xaxis_title='University',
        yaxis_title='Field',
        title_x=0.5,
        width=1200,
        height=800,
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(color='black')
    )

    # --- Save the plot ---
    fig.write_html(output_file)
    logger.info(f"Scatter plot saved as HTML: {output_file}")

    # --- Show the plot ---
    fig.show()

except Exception as e:
    logger.exception(f"Failed to create scatter plot: {e}")


# --- Cleanup ---
#del df_field_level, df_collab_field_uni, fig
gc.collect()
logger.info("Memory cleaned after scatter plot generation.")



## Trend of Top 15 Subfields by Year (Per University)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import gc

# --- Output directory ---
TREND_PLOT_DIR = Path("subfield_trend_plots")
TREND_PLOT_DIR.mkdir(parents=True, exist_ok=True)

try:
    # --- Group and filter data ---
    df_trend = (
        df_filtered
        .groupby(['Combined_University_Name', 'SUBFIELD', 'OPAL_YEAR'], as_index=False)
        .size()
        .rename(columns={'size': 'publication_count'})
    )

    top_subfields = (
        df_trend
        .groupby('SUBFIELD')['publication_count']
        .sum()
        .nlargest(15)
        .index
    )

    df_trend_top_15 = df_trend[df_trend['SUBFIELD'].isin(top_subfields)].copy()
    universities = df_trend_top_15['Combined_University_Name'].unique()
    logger.info(f"Generating trend plots for {len(universities)} universities using top 15 subfields.")

    # Create a fixed palette for 15 subfields
    palette = sns.color_palette("tab20", 15)

    for uni in universities:
        try:
            df_uni = df_trend_top_15[df_trend_top_15['Combined_University_Name'] == uni]

            plt.figure(figsize=(10, 6))
            sns.lineplot(
                data=df_uni,
                x='OPAL_YEAR',
                y='publication_count',
                hue='SUBFIELD',
                hue_order=top_subfields,  # ensures consistent color mapping
                marker="o",
                palette=palette
            )

            plt.title(f'Trend of Top 15 Subfields Over Time ({uni})')
            plt.xlabel('Year')
            plt.ylabel('Publication Count')
            plt.legend(title='Subfield', bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.xticks(range(START_YEAR, END_YEAR + 1))
            plt.grid(True)
            plt.tight_layout()

            # Save to PNG
            filename = f"{uni.replace(' ', '_').replace('/', '-')}.png"
            filepath = TREND_PLOT_DIR / filename
            plt.savefig(filepath, dpi=300)
            plt.close()
            logger.info(f"Saved trend plot: {filepath}")

        except Exception as e:
            logger.error(f"Failed to generate plot for {uni}: {e}")

except Exception as e:
    logger.exception("Error while generating subfield trend plots.")

# --- Cleanup ---
del df_trend, df_trend_top_15
gc.collect()
logger.info("Memory cleaned after subfield trend visualization.")


## Collaboration plots

## Prepare Collaboration Data Between Universities

In [ ]:
import pandas as pd
from itertools import combinations
from pathlib import Path
import logging
import gc

# --- Extract required columns ---
try:
    df_collab_base = df_filtered[['OPENALEX_ID', 'Combined_University_Name', 'DOMAIN']].drop_duplicates()
    logger.info("Collaboration base dataset created.")
except Exception as e:
    logger.exception("Failed to prepare collaboration base dataset.")
    df_collab_base = pd.DataFrame()

# --- Create university pairs grouped by publication ---
def create_uni_pairs_with_domain(group):
    try:
        universities = sorted(group['Combined_University_Name'].unique())
        domain = group['DOMAIN'].iloc[0]
        return pd.DataFrame(combinations(universities, 2), columns=['Uni1', 'Uni2']).assign(DOMAIN=domain)
    except Exception as e:
        logger.warning(f"Skipping group due to error: {e}")
        return pd.DataFrame(columns=['Uni1', 'Uni2', 'DOMAIN'])

try:
    collaboration_pairs = (
        df_collab_base
        .groupby('OPENALEX_ID')
        .apply(create_uni_pairs_with_domain)
        .reset_index(drop=True)
    )
    logger.info(f"Generated {len(collaboration_pairs)} university collaboration pairs.")
except Exception as e:
    logger.exception("Failed to generate collaboration pairs.")
    collaboration_pairs = pd.DataFrame()

# --- Aggregate total publications per university pair and domain ---
try:
    university_collaboration_domain_data = (
        collaboration_pairs
        .groupby(['Uni1', 'Uni2', 'DOMAIN'])
        .size()
        .reset_index(name='Total_Publications')
        .sort_values(by=['Uni1', 'Uni2', 'DOMAIN'])
    )
    logger.info("Aggregated university collaboration data.")
except Exception as e:
    logger.exception("Failed to aggregate university collaboration data.")
    university_collaboration_domain_data = pd.DataFrame()

# Cleanup
del df_collab_base, collaboration_pairs
gc.collect()
logger.info("Memory cleaned after data prep for collaboration networks.")


## Collaboration Network Plot - Overall

In [ ]:
## Visualize Overall University Collaboration Network
import networkx as nx
import matplotlib.pyplot as plt

try:
    # --- Collapse domain-level collaboration to overall count ---
    overall_collab = (
        university_collaboration_domain_data
        .groupby(['Uni1', 'Uni2'])
        .agg({'Total_Publications': 'sum'})
        .reset_index()
    )

    G = nx.from_pandas_edgelist(overall_collab, source='Uni1', target='Uni2', edge_attr='Total_Publications')

    # Node attributes
    total_publications = pd.concat([
        overall_collab.groupby('Uni1')['Total_Publications'].sum(),
        overall_collab.groupby('Uni2')['Total_Publications'].sum()
    ]).groupby(level=0).sum().to_dict()

    degree_centrality = nx.degree_centrality(G)
    nx.set_node_attributes(G, degree_centrality, 'degree_centrality')

    node_color = [total_publications.get(node, 0) for node in G.nodes()]
    node_size = [degree_centrality.get(node, 0) * 1000 for node in G.nodes()]
    edge_width = [G[u][v]['Total_Publications'] / 2 for u, v in G.edges()]

    # Draw
    plt.figure(figsize=(22, 28))
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, node_color=node_color, node_size=node_size, cmap=plt.cm.viridis_r, alpha=0.7)
    nx.draw_networkx_edges(G, pos, width=edge_width, alpha=0.5, edge_color='gray')

    # Edge labels
    edge_labels = nx.get_edge_attributes(G, 'Total_Publications')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='black', font_size=9)

    # Node labels
    node_labels = {node: f"{node}\nTotal: {total_publications.get(node, 0)}" for node in G.nodes()}
    nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=10, font_color='black')

    # Colorbar
    sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis_r, norm=plt.Normalize(vmin=min(node_color), vmax=max(node_color)))
    sm.set_array([])
    plt.colorbar(sm, ax=plt.gca(), label='Total Publications (Darker is Higher)')

    plt.title("University Collaboration Network (Overall)", fontsize=15)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    logger.info("Overall collaboration network plotted.")

except Exception as e:
    logger.exception("Failed to generate overall collaboration network.")


## Collaboration Network plot - per Domain

In [ ]:
domains = university_collaboration_domain_data['DOMAIN'].unique()

for domain in domains:
    try:
        domain_data = university_collaboration_domain_data[university_collaboration_domain_data['DOMAIN'] == domain]
        G = nx.from_pandas_edgelist(domain_data, source='Uni1', target='Uni2', edge_attr='Total_Publications')

        total_publications = pd.concat([
            domain_data.groupby('Uni1')['Total_Publications'].sum(),
            domain_data.groupby('Uni2')['Total_Publications'].sum()
        ]).groupby(level=0).sum().to_dict()

        degree_centrality = nx.degree_centrality(G)
        nx.set_node_attributes(G, degree_centrality, 'degree_centrality')

        node_color = [total_publications.get(node, 0) for node in G.nodes()]
        node_size = [degree_centrality.get(node, 0) * 1000 for node in G.nodes()]
        edge_width = [G[u][v]['Total_Publications'] / 2 for u, v in G.edges()]

        plt.figure(figsize=(14, 10))
        pos = nx.spring_layout(G)

        nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=node_color, cmap=plt.cm.viridis_r, alpha=0.7)
        nx.draw_networkx_edges(G, pos, width=edge_width, alpha=0.5, edge_color='gray')

        # Formatted node labels with line breaks
        node_labels = {
            node: f"{' '.join(node.split()[:2])}\n{' '.join(node.split()[2:])}\nTotal: {total_publications[node]}"
            if len(node.split()) > 2 else f"{node}\nTotal: {total_publications[node]}"
            for node in G.nodes()
        }
        nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=10, font_color='black')

        edge_labels = nx.get_edge_attributes(G, 'Total_Publications')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='black', font_size=9)

        sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis_r, norm=plt.Normalize(vmin=min(node_color), vmax=max(node_color)))
        sm.set_array([])
        plt.colorbar(sm, ax=plt.gca(), label='Total Publications (Darker is Higher)')

        plt.title(f"University Collaboration Network - Domain: {domain}", fontsize=15)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        logger.info(f"Collaboration network plotted for domain: {domain}")

    except Exception as e:
        logger.exception(f"Failed to generate collaboration network for domain: {domain}")

# Cleanup
gc.collect()
logger.info("Memory cleaned after domain collaboration networks.")


## Additional - not part of the project

### Sankey per domain with UM label (Additional - not part of the project)

In [ ]:
# Sankey diagrams with unique colors per university
import plotly.graph_objects as go
import plotly.express as px

MU_LABEL = "Maastricht University and affiliations"

# Collect all universities
all_unis = set(university_collaboration_domain_data['Uni1']).union(
    set(university_collaboration_domain_data['Uni2']))

# Large palette
palette = px.colors.qualitative.Dark24 + px.colors.qualitative.Set3 + px.colors.qualitative.Pastel1
palette_cycle = (palette * 10)

# Helper to lighten a hex / rgba color
def lighten(color, factor=0.6):
    # factor between 0 (white) and 1 (no change)
    if color.startswith("rgba"):
        r, g, b, a = eval(color.replace("rgba", ""))
        r = int(r + (255 - r) * (1 - factor))
        g = int(g + (255 - g) * (1 - factor))
        b = int(b + (255 - b) * (1 - factor))
        return f"rgba({r},{g},{b},{a})"
    if color.startswith("#"):
        color = color.lstrip("#")
        r = int(color[0:2], 16)
        g = int(color[2:4], 16)
        b = int(color[4:6], 16)
        r = int(r + (255 - r) * (1 - factor))
        g = int(g + (255 - g) * (1 - factor))
        b = int(b + (255 - b) * (1 - factor))
        return f"rgba({r},{g},{b},1)"
    return color

# Assign strong node colors
color_map = {}
i = 0
for uni in sorted(all_unis):
    if "maastricht" in uni.lower():
        color_map[uni] = "rgba(100,100,100,1)"   # MU strong gray
    else:
        color_map[uni] = palette_cycle[i]
        i += 1

# Map the MU label explicitly
color_map[MU_LABEL] = "rgba(100,100,100,1)"

for domain in domains:
    try:
        domain_data = university_collaboration_domain_data[
            university_collaboration_domain_data['DOMAIN'] == domain
            ]

        mu_rows = domain_data[
            (domain_data['Uni1'].str.contains("Maastricht University", case=False, na=False)) |
            (domain_data['Uni2'].str.contains("Maastricht University", case=False, na=False))
            ]

        if mu_rows.empty:
            logger.info(f"No MU collaborations found for domain: {domain}")
            continue

        labels = []
        label_index = {}
        node_colors = []

        def li(label):
            if label not in label_index:
                label_index[label] = len(labels)
                labels.append(label)
                node_colors.append(color_map[label])   # strong node color
            return label_index[label]

        sources = []
        targets = []
        values = []
        link_colors = []

        for _, row in mu_rows.iterrows():
            if "maastricht" in row['Uni1'].lower():
                source = MU_LABEL
                target = row['Uni2']
            else:
                source = row['Uni1']
                target = MU_LABEL

            s = li(source)
            t = li(target)

            sources.append(s)
            targets.append(t)
            values.append(row['Total_Publications'])

            # external university determines color of belt
            ext_uni = target if source == MU_LABEL else source
            base_color = color_map[ext_uni]
            link_colors.append(lighten(base_color, factor=0.45))  # lighter belt

        fig = go.Figure(data=[go.Sankey(
            node=dict(
                pad=25,
                thickness=20,
                label=labels,
                color=node_colors,        # strong color for universities
                line=dict(color="rgba(0,0,0,0)", width=0), # ← no outline
            ),
            link=dict(
                source=sources,
                target=targets,
                value=values,
                color=link_colors        # lighter belts
            )
        )])

        fig.update_layout(
            title_text=f"Sankey – {MU_LABEL} (Domain: {domain})",
            font=dict(color="black", size=12)
        )

        fig.show()
        logger.info(f"Sankey diagram plotted for domain: {domain}")

    except Exception:
        logger.exception(f"Failed to generate Sankey diagram for domain: {domain}")


## Powerpoint reports

In [ ]:
import pandas as pd
from pathlib import Path
from pptx import Presentation
from pptx.util import Inches, Pt

# =========================
# SETTINGS
# =========================
OUTPUT_DIR = Path("uni_powerpoint_slides")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
pptx_path = OUTPUT_DIR / "YUFE_Collaboration_Slides.pptx"

# =========================
# STEP 1: Compute median per field across 10 YUFE universities
# =========================
median_per_field = df_collab_field_uni.groupby('FIELD')['publication_count'].median().reset_index()
median_per_field.rename(columns={'publication_count': 'median_publications'}, inplace=True)

# Merge median into main dataframe
df_input = df_collab_field_uni.merge(median_per_field, on='FIELD', how='left')

# =========================
# STEP 2: Filter fields for each university based on median
# =========================
uni_tables = {}
all_unis = df_input['Combined_University_Name'].unique()

for uni in all_unis:
    df_uni = df_input[df_input['Combined_University_Name'] == uni].copy()
    df_uni = df_uni[df_uni['publication_count'] >= df_uni['median_publications']]
    total_pub_uni = df_uni['publication_count'].sum()
    df_uni['Share (%)'] = df_uni['publication_count'] / total_pub_uni * 100

    # # Identify partner universities per field (>= median)
    # partner_list = []
    # for _, row in df_uni.iterrows():
    #     field = row['FIELD']
    #     partners = df_input[
    #         (df_input['FIELD'] == field) &
    #         (df_input['Combined_University_Name'] != uni) &
    #         (df_input['publication_count'] >= df_input['median_publications'])
    #         ]['Combined_University_Name'].unique().tolist()
    #     partner_list.append(", ".join(partners))
    #
    # df_uni['Partner Universities'] = partner_list
    # uni_tables[uni] = df_uni[['FIELD','publication_count','Share (%)','median_publications','Partner Universities']]

    # Identify partner universities per field (>= median)
    partner_list = []
    for _, row in df_uni.iterrows():
        field = row['FIELD']
        partners_df = df_input[
            (df_input['FIELD'] == field) &
            (df_input['Combined_University_Name'] != uni) &
            (df_input['publication_count'] >= df_input['median_publications'])
            ][['Combined_University_Name','publication_count']]

        partners = [f"{p} ({c})" for p,c in zip(partners_df['Combined_University_Name'], partners_df['publication_count'])]
        partner_list.append(", ".join(partners))

    df_uni['Partner Universities'] = partner_list
    uni_tables[uni] = df_uni[['FIELD','publication_count','Share (%)','median_publications','Partner Universities']]
# =========================
# STEP 3: Create PowerPoint
# =========================
prs = Presentation()

for uni, df_uni_table in uni_tables.items():
    slide = prs.slides.add_slide(prs.slide_layouts[5])  # blank slide
    title = slide.shapes.title
    title.text = f"{uni} - Potential Collaborations (median-based)"

    rows, cols = df_uni_table.shape
    rows += 1  # for header
    table = slide.shapes.add_table(rows, cols, Inches(0.5), Inches(1.5), Inches(5), Inches(9)).table

    # Set column headers
    for j, col_name in enumerate(df_uni_table.columns):
        table.cell(0,j).text = col_name
        table.cell(0,j).text_frame.paragraphs[0].font.bold = True
        table.cell(0,j).text_frame.paragraphs[0].font.size = Pt(12)

    # Fill table
    for i in range(df_uni_table.shape[0]):
        for j in range(df_uni_table.shape[1]):
            val = df_uni_table.iloc[i,j]
            if isinstance(val,float):
                val = f"{val:.1f}"
            table.cell(i+1,j).text = str(val)
            table.cell(i+1,j).text_frame.paragraphs[0].font.size = Pt(10)

prs.save(pptx_path)
print(f"PowerPoint with 10 slides saved to {pptx_path}")
